# Pre Process the DDI Dataset

based on the github https://github.com/yardstick17/DDIExtraction/

Note: the above github is only handling the case of ddi classification - it does not look at the cases that are classified as no ddi

In [2]:
%load_ext autoreload

In [3]:
%autoreload

import os, random, pandas as pd, numpy as np
import pickle
import sys
sys.path.append('..')
# sys.path.append('../ddi_preprocess')
from relation_extraction.data import utils
import nltk
import itertools
from ast import literal_eval # to convert the string tuple form to an actual tuple
RESOURCE_PATH = "/data/medg/misc/semeval_2010/medical-data/DDICorpus"
def res(path): return os.path.join(RESOURCE_PATH, path)
from relation_extraction.data.ddi_preprocess.read_dataset import get_dataset_dataframe
from relation_extraction.data.ddi_preprocess.extract_data import extract_data_from_dataframe
from relation_extraction.data.ddi_preprocess.read_dataset import tokenize

[nltk_data] Downloading package wordnet to
[nltk_data]     /afs/csail.mit.edu/u/g/geeticka/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [164]:
#df = get_dataset_dataframe(res('Train/DrugBank/'))
#df = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/DrugBank/'))
#df = get_dataset_dataframe(res('Train/MedLine/'))
#df = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/MedLine/'))

In [4]:
df_with_extraction = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/MedLine/'), relation_extraction=True)

  9%|▉         | 3/33 [00:00<00:01, 15.31it/s]

total_files_to_read: 33  from dir:  /data/medg/misc/semeval_2010/medical-data/DDICorpus/Test/test_for_ddi_extraction_task/MedLine/


100%|██████████| 33/33 [00:00<00:00, 48.30it/s]


In [5]:
len(df_with_extraction)

388

In [6]:
df_with_extraction

,sentence_text,e1,e2,relation_type,normalized_sentence,tokenized_sentence,entity_number
0,Synergistic interaction between sunitinib and ...,sunitinib,docetaxel,int,Synergistic_bf interaction_bf DRUG OTHER_DRUG ...,Synergistic interaction between sunitinib and ...,"([3], [5])"
1,This study was aimed to investigate the effica...,sunitinib,sunitinib,none,This_bf study_bf aimed_bf investigate_bf effic...,This study was aimed to investigate the effica...,"([12], [12])"
2,This study was aimed to investigate the effica...,sunitinib,docetaxel,none,This_bf study_bf aimed_bf investigate_bf effic...,This study was aimed to investigate the effica...,"([12], [23])"
3,This study was aimed to investigate the effica...,sunitinib,docetaxel,none,This_bf study_bf aimed_bf investigate_bf effic...,This study was aimed to investigate the effica...,"([12], [23])"
4,NSCLC cell lines with EGFR T790M mutation and ...,sunitinib,docetaxel,none,NSCLC_bf cell_bf lines_bf EGFR_bf T790M_bf mut...,NSCLC cell lines with EGFR T790 M mutation and...,"([17], [19])"
5,Sunitinib exhibited dose-dependent growth inhi...,Sunitinib,docetaxel,none,DRUG exhibited_be dose-dependent_be growth_be ...,Sunitinib exhibited dose - dependent growth in...,"([0], [20])"
6,Although single or concurrent use of sunitinib...,sunitinib,docetaxel,effect,Although_bf single_bf concurrent_bf use_bf DRU...,Although single or concurrent use of sunitinib...,"([6], [8])"
7,When cells were exposed to docetaxel followed ...,docetaxel,sunitinib,effect,When_bf cells_bf exposed_bf DRUG followed_be O...,When cells were exposed to docetaxel followed ...,"([5], [8])"
8,The molecular basis of this synergism is that ...,docetaxel,sunitinib,effect,The_bf molecular_bf basis_bf synergism_bf sign...,The molecular basis of this synergism is that ...,"([16], [26])"
9,Sunitinib as a single agent exhibits anti-prol...,Sunitinib,docetaxel,none,DRUG single_be agent_be exhibits_be anti-proli...,Sunitinib as a single agent exhibits anti - pr...,"([0], [30])"


## Get information about the empty entity rows

In [189]:
def get_empty_entity_rows(df):
    empty_entity_rows = []
    def find_empty_entity_number(row):
        entity_number = row.entity_number
        e1 = entity_number[0]
        e2 = entity_number[1]
        if not e1 or not e2:
            empty_entity_rows.append(row.row_num)
    temp_df = df.copy()
    temp_df.insert(0, 'row_num', range(0, len(temp_df)))
    temp_df.apply(find_empty_entity_number, axis=1)
    return empty_entity_rows

def get_empty_rows_array(empty_entity_rows, df):
    empty_rows_array = []
    for index in empty_entity_rows:
        e1 = df.iloc[index].e1
        e2 = df.iloc[index].e2
        sentence_text = df.iloc[index].sentence_text
        tokenized_sentence = df.iloc[index].tokenized_sentence
        entity_number = df.iloc[index].entity_number
        empty_rows_array.append([index, e1, e2, sentence_text, tokenized_sentence, entity_number])
    new_df = pd.DataFrame(data=empty_rows_array,    # values
             columns=['index_original', 'e1', 'e2', 'sentence_text', 'tokenized_sentence', 'entity_number'])
    return empty_rows_array, new_df

In [190]:
def get_empty_vals(df):
    empty_entity_rows = get_empty_entity_rows(df)
    empty_rows_array, new_df = get_empty_rows_array(empty_entity_rows, df)
    return empty_rows_array, new_df

Either do a character based detection for the above, or try to handle the cases where you have plurals occuring as well as (1) or something in the text. Can define a "loose equality" method to handle this. 

In [191]:
# let word 1 be entity
def loose_equality(word1, word2):
    # first check up to which character the words are different, and if that is less than 3/4 through the 
    # length of that word, discard it
    i=0 
    for c1, c2 in zip(word1, word2):
        if c1 == c2:
            i+=1
        else:
            break
    # if the beginning doesn't match, forget it
    if i < 3.0/4.0 * len(word1):
        return False
    else:
        edit_distance = nltk.edit_distance(word1, word2)
        if edit_distance < 3.0/4.0 * len(word1):
            return True
        else:
            return False

In [192]:
def fix_entity_number(row):
    tokenized_sentence = row.tokenized_sentence
    e1 = tokenize(row.e1)
    e2 = tokenize(row.e2)
    e1_limit = []
    e2_limit = []

    e1 = e1.split()
    idx_e1 = 0
    e1_found = False
    
    e2 = e2.split()
    idx_e2 = 0
    e2_found = False
    
    curr_entity_nums = row.old_entity_number
    curr_e1_nums = curr_entity_nums[0]
    curr_e2_nums = curr_entity_nums[1]
    
    idx_sent = 0
    for word in tokenized_sentence.split():
        if not curr_e1_nums: # only if e1 is null do this
            if idx_e1 < len(e1) and loose_equality(e1[idx_e1].lower(), word.lower()):
                e1_limit.append(idx_sent)
                idx_e1 += 1
                e1_found = True
            elif idx_e1 < len(e1) and e1_found is True and loose_equality(e1[idx_e1], word.lower()):
                idx_e1 = 0
                e1_found = False
                e1_limit = []
        if not curr_e2_nums: # only if e2 is null do this
            if idx_e2 < len(e2) and loose_equality(e2[idx_e2].lower(), word.lower()):
                e2_limit.append(idx_sent)
                idx_e2 += 1
                e2_found = True
            elif idx_e2 < len(e2) and e2_found is True and loose_equality(e2[idx_e2].lower(), word.lower()):
                idx_e2 = 0
                e2_found = False
                e2_limit = []
        idx_sent += 1
    # only return what is not empty
    if not curr_e1_nums: 
        e1_return = e1_limit
    else:
        e1_return = curr_e1_nums
    if not curr_e2_nums:
        e2_return = e2_limit
    else:
        e2_return = curr_e2_nums
    return e1_return, e2_return

In [193]:
# given a dataframe (without the empty vals), fix them
# returns only a dataframe with the empty values fixed
def fix_empty_vals(df):
    empty_rows_array, new_df = get_empty_vals(df)
    new_df = new_df.rename(columns={'entity_number': 'old_entity_number'})
    new_df['entity_number'] = new_df.apply(fix_entity_number, axis=1)
    return new_df

### First round of fixing

In [194]:
df_with_fixed_vals = fix_empty_vals(df_with_extraction)

In [195]:
df_with_fixed_vals

,index_original,e1,e2,sentence_text,tokenized_sentence,old_entity_number,entity_number
0,101,Ticagrelor,Brilinta,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta)--better than clopidogre...,"([0], [])","([0], [])"
1,104,Brilinta,clopidogrel,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta)--better than clopidogre...,"([], [4])","([], [4])"
2,105,Brilinta,Plavix,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta)--better than clopidogre...,"([], [6])","([], [6])"
3,140,Calcium,nonheme iron,Calcium does not inhibit the absorption of 5 m...,Calcium does not inhibit the absorption of 5 m...,"([0], [])","([0], [10, 13])"
4,142,nonheme iron,heme iron,Calcium does not inhibit the absorption of 5 m...,Calcium does not inhibit the absorption of 5 m...,"([], [12, 13])","([10, 13], [12, 13])"
5,143,Calcium,nonheme iron,Calcium is the only known component in the die...,Calcium is the only known component in the die...,"([0], [])","([0], [15, 18])"
6,145,nonheme iron,heme iron,Calcium is the only known component in the die...,Calcium is the only known component in the die...,"([], [17, 18])","([15, 18], [17, 18])"
7,149,calcium,nonheme iron,Our objective was to establish potential effec...,Our objective was to establish potential effec...,"([8], [])","([8], [12, 15])"
8,151,nonheme iron,heme iron,Our objective was to establish potential effec...,Our objective was to establish potential effec...,"([], [14, 15])","([12, 15], [14, 15])"
9,161,calcium,nonheme iron,"In conclusion, we demonstrated an isolated eff...","In conclusion , we demonstrated an isolated ef...","([9], [])","([9], [23, 34])"


In [196]:
_, second_round_of_empty_vals = get_empty_vals(df_with_fixed_vals)

In [197]:
second_round_of_empty_vals

,index_original,e1,e2,sentence_text,tokenized_sentence,entity_number
0,0,Ticagrelor,Brilinta,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta)--better than clopidogre...,"([0], [])"
1,1,Brilinta,clopidogrel,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta)--better than clopidogre...,"([], [4])"
2,2,Brilinta,Plavix,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta)--better than clopidogre...,"([], [6])"
3,18,AAV2,heparinase III,AAV2-mediated retinal transduction is improved...,AAV2-mediated retinal transduction is improved...,"([], [10, 11])"
4,19,AAV2,chondroitin ABC lyase,AAV2-mediated retinal transduction is improved...,AAV2-mediated retinal transduction is improved...,"([], [13, 14, 15])"


In [198]:
#second_round_of_empty_vals.at[0, 'entity_number'] = ([15, 16], [39])

In [199]:
second_round_of_empty_vals.loc[0]['sentence_text']

'Ticagrelor (Brilinta)--better than clopidogrel (Plavix)'

In [200]:
second_round_of_empty_vals.loc[0]['tokenized_sentence']

'Ticagrelor ( Brilinta)--better than clopidogrel ( Plavix )'

### Second round of fixing (not needed for DrugBank Test Data)

In [201]:
def update(second_round_of_empty_vals, data):
    def update_vals(row, data=data):
        if row.index_original == data['index_original']:
            row.tokenized_sentence = data['tokenized_sentence']
            row.entity_number = data['entity_number']
        return row

    second_round_of_empty_vals = second_round_of_empty_vals.apply(update_vals, axis=1)
    return second_round_of_empty_vals

In [202]:
# Below is for the test data of MedLine for extraction case
data = [{    'index_original' :  0, 
             'tokenized_sentence': 'Ticagrelor ( Brilinta ) -- better than clopidogrel ( Plavix )',
             'entity_number': ([0], [2])
    
        }, 
        {    'index_original' :  1, 
             'tokenized_sentence': 'Ticagrelor ( Brilinta ) -- better than clopidogrel ( Plavix )',
             'entity_number': ([2], [7])
    
        },
        {    'index_original' :  2, 
             'tokenized_sentence': 'Ticagrelor ( Brilinta ) -- better than clopidogrel ( Plavix )',
             'entity_number': ([2], [9])
    
        }, 
        {    'index_original' :  18, 
             'tokenized_sentence': 'AAV2 - mediated retinal transduction is improved by co - injection of heparinase III or chondroitin ABC lyase .',
             'entity_number': ([0], [12, 13])
    
        }, 
        {    'index_original' :  19, 
             'tokenized_sentence': 'AAV2 - mediated retinal transduction is improved by co - injection of heparinase III or chondroitin ABC lyase .',
             'entity_number': ([0], [15, 16, 17])
            
        }]

In [203]:
# #Below is for the train data in MedLine for extraction case
# data = [{   'index_original' :  9, 
#             'tokenized_sentence': 'Treatment of toxin A with [(14)C] - diethyl pyrocarbonate revealed concentration dependent labelling of histidine residues on the toxin molecules .',
#             'entity_number': ([2,3], [7,8])
#         }, 
#        {
#            'index_original': 10, 
#            'tokenized_sentence': 'The effects of ruthenium red ( RR ) on inositol 1,4,5-trisphosphate ( InsP(3) )-induced responses were studied in rat bone marrow megakaryocytes with the patch - clamp whole - cell recording technique in combination with fura-2 microfluorometry .',
#            'entity_number': ([3,4], [12])
#        }, 
#        {
#            'index_original': 11, 
#            'tokenized_sentence': 'The effects of ruthenium red ( RR ) on inositol 1,4,5-trisphosphate ( InsP(3) )-induced responses were studied in rat bone marrow megakaryocytes with the patch - clamp whole - cell recording technique in combination with fura-2 microfluorometry .',
#            'entity_number': ([6], [12])
#        },
#        {
#            'index_original': 12, 
#            'tokenized_sentence': 'The effects of ruthenium red ( RR ) on inositol 1,4,5-trisphosphate ( InsP(3) )-induced responses were studied in rat bone marrow megakaryocytes with the patch - clamp whole - cell recording technique in combination with fura-2 microfluorometry .',
#            'entity_number': ([9,10], [12])
#        },
#        {
#            'index_original': 13, 
#            'tokenized_sentence': 'Pretreatment of megakaryocytes with extracellular RR ( 50 microM ) also inhibited InsP(3) - induced responses .',
#            'entity_number': ([5], [12])
#        },
#        {
#            'index_original': 14, 
#            'tokenized_sentence': 'In contrast , in isolated single pancreatic acinar cells , RR had no effect on InsP(3) - induced responses .',
#            'entity_number': ([10], [15])
#        },
#         {
#            'index_original': 15, 
#            'tokenized_sentence': 'In addition , we have shown that RR is a useful pharmacological tool with which to examine the InsP(3) - mediated responses of megakaryocytes .',
#            'entity_number': ([7], [18])
#        },
#         {
#            'index_original': 16, 
#            'tokenized_sentence': 'The effects of DCG - IV and L - CCG - 1 upon phencyclidine ( PCP ) - induced locomotion and behavioral changes in mice .',
#            'entity_number': ([3,4,5], [15])
#        },
#         {
#            'index_original': 17, 
#            'tokenized_sentence': 'The effects of DCG - IV and L - CCG - 1 upon phencyclidine ( PCP ) - induced locomotion and behavioral changes in mice .',
#            'entity_number': ([7,8,9,10,11], [15])
#        },
#         {
#            'index_original': 18, 
#            'tokenized_sentence': 'The effects of DCG - IV and L - CCG - 1 upon phencyclidine ( PCP ) - induced locomotion and behavioral changes in mice .',
#            'entity_number': ([13], [15])
#        },
#         {
#            'index_original': 24, 
#            'tokenized_sentence': 'The percentage of neurons hyperpolarized by mu - selective opioids , delta(1) - selective opioids , and kappa - selective opioids was significantly reduced when 1 % but not <',
#            'entity_number': ([6,7,8,9], [11,12,13,14])
#        },
#         {
#            'index_original': 26, 
#            'tokenized_sentence': 'The percentage of neurons hyperpolarized by mu - selective opioids , delta(1) - selective opioids , and kappa - selective opioids was significantly reduced when 1 % but not <',
#            'entity_number': ([11,12,13,14], [17,18,19,20])
#        },
#         {
#            'index_original': 28, 
#            'tokenized_sentence': 'Therapeutic drug monitoring can avoid iatrogenic alterations caused by 99mTc - methylene diphosphonate ( MDP ) - gentamicin interaction .',
#            'entity_number': ([9,10,11,12], [17])
#        },
#         {
#            'index_original': 29, 
#            'tokenized_sentence': 'The intestinal absorption of arsenate ( As(V) ) has been investigated in the chick by means of the in situ ligated duodenal loop technique .',
#            'entity_number': ([4], [6])
#        }
#        ]

In [204]:
# #Below is for the train data in DrugBank for extraction case
# data = [{   'index_original' :  302, 
#             'tokenized_sentence': 'In Study 1 , patients with colorectal cancer were given irinotecan / 5-FU / leucovorin ( bolus - IFL ) with or without AVASTIN .',
#             'entity_number': ([10], [12])
#         }, 
#        {
#            'index_original': 305, 
#            'tokenized_sentence': 'In Study 1 , patients with colorectal cancer were given irinotecan / 5-FU / leucovorin ( bolus - IFL ) with or without AVASTIN .',
#            'entity_number': ([12], [14])
#        }, 
#        {
#            'index_original': 306, 
#            'tokenized_sentence': 'In Study 1 , patients with colorectal cancer were given irinotecan / 5-FU / leucovorin ( bolus - IFL ) with or without AVASTIN .',
#            'entity_number': ([12], [23])
#        },
#        {   'index_original': 324, 
#            'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline ) -- commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',
#            'entity_number': ([15, 16], [39])
           
#        }, 
#        {
#            'index_original': 325, 
#            'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline ) -- commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',
#            'entity_number': ([20], [39])
#        },
#        {
#            'index_original': 326, 
#            'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline ) -- commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',
#            'entity_number': ([24, 25], [39])
#        },
#        {
#            'index_original': 327, 
#            'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline ) -- commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',
#            'entity_number': ([29], [39])
#        },
#        {
#            'index_original': 328, 
#            'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline ) -- commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',
#            'entity_number': ([34, 35], [39])
#        },
#        {
#            'index_original': 357, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([0], [12])
#        },
#        {
#            'index_original': 358, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([2], [12])
#        },
#        {
#            'index_original': 359, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([12], [18])
#        },
#        {
#            'index_original': 360, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([12], [21])
#        },
#        {
#            'index_original': 361, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([12], [25, 26])
#        },
#        {
#            'index_original': 362, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([12], [31])
#        },
#        {
#            'index_original': 363, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([12], [21])
#        },
#        {
#            'index_original': 364, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([12], [38])
#        },
#        {
#            'index_original': 365, 
#            'tokenized_sentence': 'Antihypertensives : Amiodarone should be used with caution in patients receiving - receptor blocking agents ( e.g. , propranolol , a CYP3A4 inhibitor ) or calcium channel antagonists ( e.g. , verapamil , a CYP3A4 substrate , and diltiazem , a CYP3A4 inhibitor ) because of the possible potentiation of bradycardia , sinus arrest , and AV block ;',
#            'entity_number': ([12], [21])
#        },
#        {
#            'index_original': 366, 
#            'tokenized_sentence': 'Cholestyramine increases enterohepatic elimination of amiodarone and may reduce its serum levels and t1 / 2 .',
#            'entity_number': ([0], [13])
#        },
#        {
#            'index_original': 367, 
#            'tokenized_sentence': 'Cholestyramine increases enterohepatic elimination of amiodarone and may reduce its serum levels and t1 / 2 .',
#            'entity_number': ([5], [13])
#        }]

In [205]:
# Below is for the train data in DrugBank for classification case
# data = [{   'index_original' :  26, 
#             'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline )--commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',  
#             'entity_number': ([15,16], [39])
#         }, 
#        {
#            'index_original': 27, 
#            'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline )--commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',
#            'entity_number': ([20], [39])
#        }, 
#        {   'index_original': 28, 
#            'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline )--commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',
#            'entity_number': ([24, 25], [39])
           
#        }, 
#        {
#            'index_original': 29, 
#            'tokenized_sentence': 'The concurrent use of two or more drugs with anticholinergic activity -- such as an antipsychotic drug ( eg , chlorpromazine ) , an antiparkinsonian drug ( eg , trihexyphenidyl ) , and/or a tricyclic antidepressant ( eg , amitriptyline )--commonly results in excessive anticholinergic effects , including dry mouth and associated dental complications , blurred vision , and , in patients exposed to high temperature and humidity , hyperpyrexia .',
#            'entity_number': ([29], [39])
#        }]

In [206]:
#second_round_of_empty_vals.values

In [207]:
# # Below is for the train data of MedLine for classification case
# data = [{   'index_original' :  2, 
#             'tokenized_sentence': 'Pretreatment of megakaryocytes with extracellular RR ( 50 microM ) also inhibited InsP(3) - induced responses .',
#             'entity_number': ([5], [12])
#         }, 
#        {
#            'index_original': 4, 
#            'tokenized_sentence': 'Therapeutic drug monitoring can avoid iatrogenic alterations caused by 99mTc - methylene diphosphonate ( MDP ) - gentamicin interaction .',
#            'entity_number': ([9, 10, 11, 12], [17])
#        }]

In [208]:
# # Below is for the test data of MedLine for classification case
# data = [{    'index_original' :  3, 
#              'tokenized_sentence': 'AAV2 - mediated retinal transduction is improved by co - injection of heparinase III or chondroitin ABC lyase .',
#              'entity_number': ([0], [12, 13])
    
#         }, 
#         {    'index_original' :  4, 
#              'tokenized_sentence': 'AAV2 - mediated retinal transduction is improved by co - injection of heparinase III or chondroitin ABC lyase .',
#              'entity_number': ([0], [15, 16, 17])
            
#         }]

In [209]:
for d in data:
    second_round_of_empty_vals = update(second_round_of_empty_vals, d)


In [210]:
second_round_of_empty_vals

,index_original,e1,e2,sentence_text,tokenized_sentence,entity_number
0,0,Ticagrelor,Brilinta,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta ) -- better than clopido...,"([0], [2])"
1,1,Brilinta,clopidogrel,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta ) -- better than clopido...,"([2], [7])"
2,2,Brilinta,Plavix,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta ) -- better than clopido...,"([2], [9])"
3,18,AAV2,heparinase III,AAV2-mediated retinal transduction is improved...,AAV2 - mediated retinal transduction is improv...,"([0], [12, 13])"
4,19,AAV2,chondroitin ABC lyase,AAV2-mediated retinal transduction is improved...,AAV2 - mediated retinal transduction is improv...,"([0], [15, 16, 17])"


In [211]:
def update_df_with_fixed_vals(fromdf, todf):
    todf = todf.copy()
    for i in range(0, len(fromdf)):
        row = fromdf.iloc[i]
        idx = row.index_original
        entity_number = row.entity_number
        tokenized_sentence = row.tokenized_sentence
        todf.at[idx, 'entity_number'] = entity_number
        todf.at[idx, 'tokenized_sentence'] = tokenized_sentence
    return todf

#### Update the first round of df

In [212]:
df_with_fixed_vals = update_df_with_fixed_vals(second_round_of_empty_vals, df_with_fixed_vals)

In [213]:
df_with_fixed_vals

,index_original,e1,e2,sentence_text,tokenized_sentence,old_entity_number,entity_number
0,101,Ticagrelor,Brilinta,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta ) -- better than clopido...,"([0], [])","([0], [2])"
1,104,Brilinta,clopidogrel,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta ) -- better than clopido...,"([], [4])","([2], [7])"
2,105,Brilinta,Plavix,Ticagrelor (Brilinta)--better than clopidogrel...,Ticagrelor ( Brilinta ) -- better than clopido...,"([], [6])","([2], [9])"
3,140,Calcium,nonheme iron,Calcium does not inhibit the absorption of 5 m...,Calcium does not inhibit the absorption of 5 m...,"([0], [])","([0], [10, 13])"
4,142,nonheme iron,heme iron,Calcium does not inhibit the absorption of 5 m...,Calcium does not inhibit the absorption of 5 m...,"([], [12, 13])","([10, 13], [12, 13])"
5,143,Calcium,nonheme iron,Calcium is the only known component in the die...,Calcium is the only known component in the die...,"([0], [])","([0], [15, 18])"
6,145,nonheme iron,heme iron,Calcium is the only known component in the die...,Calcium is the only known component in the die...,"([], [17, 18])","([15, 18], [17, 18])"
7,149,calcium,nonheme iron,Our objective was to establish potential effec...,Our objective was to establish potential effec...,"([8], [])","([8], [12, 15])"
8,151,nonheme iron,heme iron,Our objective was to establish potential effec...,Our objective was to establish potential effec...,"([], [14, 15])","([12, 15], [14, 15])"
9,161,calcium,nonheme iron,"In conclusion, we demonstrated an isolated eff...","In conclusion , we demonstrated an isolated ef...","([9], [])","([9], [23, 34])"


#### Update the original dataframe

In [214]:
df_copy = update_df_with_fixed_vals(df_with_fixed_vals, df_with_extraction)

Now I can write the above dataframe as the training data

In [216]:
#df_copy.to_csv(res('pre-processed/extraction/Train/DrugBank/train-drugbank.csv'), encoding='utf-8', index=False)
# this is only relevant to the non NER task
#df_copy.to_csv(res('pre-processed/extraction/Test/DrugBank/test-drugbank.csv'), encoding='utf-8', index=False)
#df_copy.to_csv(res('pre-processed/extraction/Train/MedLine/train-medline.csv'), encoding='utf-8', index=False)
#df_copy.to_csv(res('pre-processed/extraction/Test/MedLine/test-medline.csv'), encoding='utf-8', index=False)

In [139]:
#df_copy_copy = pd.read_csv(res('pre-processed/Test/MedLine/test-medline.csv'))

In [143]:
#pd.DataFrame.equals(df_copy, df_copy_copy)

In [144]:
#df_copy_copy

In [145]:
#df_copy

In [146]:
#df_copy.sort_index(axis=1) == df_copy_copy.sort_index(axis=1)

#### Write the files into .txt format

In Train DrugBank, there is one sentence without a pair. Did ```find . -name "*.xml" | xargs grep "Drug/Laboratory Test Interactions:"``` for that

In [251]:
directory = 'pre-processed/extraction/Test/MedLine/'
filename = 'test-medline'

In [252]:
df_copy = pd.read_csv(res(directory+filename+'.csv'))

In [253]:
df_copy['relation_type'].unique()

array(['int', 'none', 'effect', 'mechanism', 'advise'], dtype=object)

In [254]:
relation_dict = {'advise': 0, 'effect': 1, 'mechanism': 2, 'int': 3, 'none': 4} 

Relation, e1, e2, sentence is the order of writing the txt file

In [255]:
null_row = df_copy[df_copy["relation_type"].isnull()]

In [256]:
if null_row.empty:
    idx_null_row = None
else:
    idx_null_row = null_row.index.values[0]

In [257]:
with open(res(directory + filename+'.txt'), 'w') as outfile:
    for i in range(0, len(df_copy)):
        if idx_null_row is not None and i == idx_null_row:
            continue
        row = df_copy.iloc[i]
        relation = relation_dict[row.relation_type]
        entity_number = literal_eval(row.entity_number)
        # literal eval is needed because the tuple is being stored as a string
        e1 = entity_number[0]
        e2 = entity_number[1]
        tokenized_sentence = row.tokenized_sentence
        outfile.write(str(relation) + " " + str(e1[0]) + " " + str(e1[-1]) + " " + 
                      str(e2[0]) + " " + str(e2[-1]) + " " + tokenized_sentence + "\n")
    outfile.close()

There is a nan in the relation in the above, for some strange reason. 

Relation, e1, e2, sentence is the order of writing the txt file

### Combine the Train/Test data of MedLine and DrugBank

In [260]:
directory2 = 'pre-processed/extraction/Train/MedLine/'
file2 = 'train-medline'
directory1 = 'pre-processed/extraction/Train/DrugBank/'
file1 = 'train-drugbank'
outfile = 'pre-processed/extraction/train.txt'

In [261]:
# https://stackoverflow.com/questions/13613336/python-concatenate-text-files
filenames = [res(directory1 + file1+'.txt'), res(directory2 + file2+'.txt')]
with open(res(outfile), 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)